## 1. Import Relevant Packages

In [1]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import *
from scipy import stats
import datetime as dt

import matplotlib.pyplot as plt
import statsmodels.api as sm

## 2. Daily returns

### Load daily returns

In [2]:
# Loading daily dataframe
d_ret = pd.read_csv('dailyreturns_1931.csv', low_memory=False)
#d_ret = pd.read_csv('dailyreturns.csv', low_memory=False)

In [3]:
start_date = '1931-01-01'

In [4]:
# Defining result starting data
d_ret = d_ret[d_ret['date'] >= start_date]

In [5]:
# Make all column names lower case
d_ret.columns = d_ret.columns.str.lower()

In [6]:
# Dropping irrelevant columns
d_ret.drop(['shrcd', 'exchcd'], inplace = True, axis=1)

In [7]:
d_ret = d_ret.rename(columns={'date':'daily_date'})

In [8]:
# Making date column to datetime
d_ret1 = d_ret.copy()
d_ret1['daily_date'] = pd.to_datetime(d_ret1.daily_date)

In [9]:
# Make new column for merging with monthly rank
d_ret1['date_merge'] = d_ret1['daily_date']
d_ret1['date_merge'] = d_ret1['date_merge'].dt.strftime("%Y/%m")

In [10]:
d_ret1.groupby('daily_date').head()

,permno,daily_date,siccd,dlret,ret,date_merge
0,10000,1986-01-07,3990,NaN,C,1986/01
1,10000,1986-01-08,3990,NaN,-0.024390,1986/01
2,10000,1986-01-09,3990,NaN,0.000000,1986/01
3,10000,1986-01-10,3990,NaN,0.000000,1986/01
4,10000,1986-01-13,3990,NaN,0.050000,1986/01
...,...,...,...,...,...,...
44095684,60186,2001-09-11,5047,NaN,NaN,2001/09
45023917,61664,1977-01-15,6723,NaN,NaN,1977/01
49785525,71731,1978-11-26,7512,NaN,NaN,1978/11
52708453,76225,1972-12-28,3911,NaN,NaN,1972/12


In [11]:
d_ret.sort_values('daily_date')

,permno,daily_date,siccd,dlret,ret
11556887,15632,1931/01/02,4000,NaN,NaN
9969522,14592,1931/01/02,2830,NaN,0.020000
11002855,15253,1931/01/02,3710,NaN,0.000000
12174413,16117,1931/01/02,4990,NaN,-0.016204
16316312,19473,1931/01/02,2910,NaN,0.066667
...,...,...,...,...,...
69698510,87725,2020/12/31,3315,A,0.000899
28956795,35044,2020/12/31,6021,A,0.011927
47412163,65700,2020/12/31,3274,A,-0.012132
10333625,14807,2020/12/31,9999,A,-0.029787


In [12]:
# Save as CSV
d_ret1.to_csv('daily_1931.csv', index=False)

In [13]:
d_ret1[d_ret1['permno'] == 91103].tail(30)

,permno,daily_date,siccd,dlret,ret,date_merge
73710400,91103,2020-11-18,4512,NaN,0.011084,2020/11
73710401,91103,2020-11-19,4512,NaN,-0.002680,2020/11
73710402,91103,2020-11-20,4512,NaN,-0.025403,2020/11
73710403,91103,2020-11-23,4512,NaN,0.025815,2020/11
73710404,91103,2020-11-24,4512,NaN,0.098461,2020/11
73710405,91103,2020-11-25,4512,NaN,0.015125,2020/11
73710406,91103,2020-11-27,4512,NaN,-0.007450,2020/11
73710407,91103,2020-11-30,4512,NaN,-0.005519,2020/11
73710408,91103,2020-12-01,4512,NaN,0.005993,2020/12
73710409,91103,2020-12-02,4512,NaN,0.031112,2020/12


### Load holding period DataFrame

In [55]:
# Taking the relevant columns from the holding df 
h_to_daily = pd.read_csv('holding_df.csv', low_memory=False)
h_month = h_to_daily.copy()
h_to_daily = h_to_daily[['permno', 'hdate2', 'momr']]
h_to_daily.rename(columns={'hdate2':'monthly_date'}, inplace=True)

In [56]:
h_month

,permno,form_date,momr,hdate1,hdate2,date,ret
0,12562,1930-10-31,10.0,1930-12-01,1930-12-31,1930-12-31,0.003322
1,17021,1930-10-31,10.0,1930-12-01,1930-12-31,1930-12-31,-0.157895
2,12474,1930-10-31,10.0,1930-12-01,1930-12-31,1930-12-31,-0.049107
3,15050,1930-10-31,10.0,1930-12-01,1930-12-31,1930-12-31,-0.022026
4,11690,1930-10-31,10.0,1930-12-01,1930-12-31,1930-12-31,0.106061
...,...,...,...,...,...,...,...
540495,13356,2020-10-31,1.0,2020-12-01,2020-12-31,2020-12-31,0.154506
540496,38703,2020-10-31,1.0,2020-12-01,2020-12-31,2020-12-31,0.103473
540497,19561,2020-10-31,1.0,2020-12-01,2020-12-31,2020-12-31,0.015899
540498,91103,2020-10-31,1.0,2020-12-01,2020-12-31,2020-12-31,-0.039956


In [16]:
# To Python datatime fomat
h_to_daily['monthly_date'] = pd.to_datetime(h_to_daily.monthly_date)

In [17]:
# Create column with date for merging 
h_to_daily['date_merge'] = h_to_daily['monthly_date']
h_to_daily['date_merge'] = h_to_daily['date_merge'].dt.strftime("%Y/%m")

In [18]:
h_to_daily.groupby()

,permno,monthly_date,momr,date_merge
0,12562,1930-12-31,10.0,1930/12
1,17021,1930-12-31,10.0,1930/12
2,12474,1930-12-31,10.0,1930/12
3,15050,1930-12-31,10.0,1930/12
4,11690,1930-12-31,10.0,1930/12
...,...,...,...,...
540495,13356,2020-12-31,1.0,2020/12
540496,38703,2020-12-31,1.0,2020/12
540497,19561,2020-12-31,1.0,2020/12
540498,91103,2020-12-31,1.0,2020/12


In [19]:
# Merge on date and permnos to get daily returns
daily = pd.merge(d_ret1, h_to_daily, on=['date_merge', 'permno'], how='left')

In [20]:
daily['ret'] = pd.to_numeric(daily['ret'], errors='coerce') 

In [21]:
daily.head()

,permno,daily_date,siccd,dlret,ret,date_merge,monthly_date,momr
0,10000,1986-01-07,3990,NaN,NaN,1986/01,NaT,NaN
1,10000,1986-01-08,3990,NaN,-0.02439,1986/01,NaT,NaN
2,10000,1986-01-09,3990,NaN,0.00000,1986/01,NaT,NaN
3,10000,1986-01-10,3990,NaN,0.00000,1986/01,NaT,NaN
4,10000,1986-01-13,3990,NaN,0.05000,1986/01,NaT,NaN


In [22]:
daily

,permno,daily_date,siccd,dlret,ret,date_merge,monthly_date,momr
0,10000,1986-01-07,3990,NaN,NaN,1986/01,NaT,NaN
1,10000,1986-01-08,3990,NaN,-0.024390,1986/01,NaT,NaN
2,10000,1986-01-09,3990,NaN,0.000000,1986/01,NaT,NaN
3,10000,1986-01-10,3990,NaN,0.000000,1986/01,NaT,NaN
4,10000,1986-01-13,3990,NaN,0.050000,1986/01,NaT,NaN
...,...,...,...,...,...,...,...,...
75696491,93436,2020-12-24,9999,NaN,0.024444,2020/12,2020-12-31,10.0
75696492,93436,2020-12-28,9999,NaN,0.002901,2020/12,2020-12-31,10.0
75696493,93436,2020-12-29,9999,NaN,0.003465,2020/12,2020-12-31,10.0
75696494,93436,2020-12-30,9999,NaN,0.043229,2020/12,2020-12-31,10.0


In [23]:
# Exclude when they are not in universe
daily_filtered = daily[daily['momr'].notna()]

In [24]:
daily_filtered.groupby(['daily_date']).head()

,permno,daily_date,siccd,dlret,ret,date_merge,monthly_date,momr
19064,10006,1931-01-02,3740,NaN,0.022727,1931/01,1931-01-31,3.0
19065,10006,1931-01-03,3740,NaN,0.075556,1931/01,1931-01-31,3.0
19066,10006,1931-01-05,3740,NaN,0.024793,1931/01,1931-01-31,3.0
19067,10006,1931-01-06,3740,NaN,0.036290,1931/01,1931-01-31,3.0
19068,10006,1931-01-07,3740,NaN,0.027237,1931/01,1931-01-31,3.0
...,...,...,...,...,...,...,...,...
20811137,23473,1985-09-27,6711,NaN,NaN,1985/09,1985-09-30,10.0
31146131,38762,2012-10-29,4931,NaN,NaN,2012/10,2012-10-31,4.0
41849858,56274,2012-10-29,2011,NaN,NaN,2012/10,2012-10-31,2.0
43242834,58819,2012-10-29,4931,NaN,NaN,2012/10,2012-10-31,4.0


In [25]:
# Ensure ret is float value
daily_filtered['ret'] = daily_filtered['ret'].astype(float)

<ipython-input-25-5c7418d2152a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_filtered['ret'] = daily_filtered['ret'].astype(float)


In [26]:
# Create datafame for portfolio return calculation
port_d_return = pd.DataFrame(daily_filtered[['daily_date', 'siccd', 'ret', 'momr']])

In [27]:
# Calculating mean (portfolio returns) for a given date, momentum ranking
port_d_return = port_d_return.groupby(['daily_date','momr'])['ret'].mean().reset_index()
port_d_return_t = port_d_return.copy()

In [28]:
port_d_return_t

,daily_date,momr,ret
0,1931-01-02,1.0,0.108129
1,1931-01-02,2.0,0.084084
2,1931-01-02,3.0,0.029738
3,1931-01-02,4.0,0.062567
4,1931-01-02,5.0,0.059118
...,...,...,...
235619,2020-12-31,6.0,0.008454
235620,2020-12-31,7.0,0.007420
235621,2020-12-31,8.0,0.005092
235622,2020-12-31,9.0,0.003129


In [29]:
# Check mean daily return
port_d_return_t.groupby('momr')['ret'].describe()[['count','mean','std']].reset_index()

,momr,count,mean,std
0,1.0,23561.0,0.000463,0.015382
1,2.0,23561.0,0.000532,0.013005
2,3.0,23561.0,0.000546,0.012137
3,4.0,23561.0,0.000556,0.011396
4,5.0,23561.0,0.000528,0.011068
5,6.0,23561.0,0.000565,0.010949
6,7.0,23561.0,0.000561,0.010978
7,8.0,23561.0,0.000597,0.011333
8,9.0,23561.0,0.000674,0.012077
9,10.0,23561.0,0.000837,0.014622


In [30]:
# Transpose portfolio layout to have columns as portfolio returns
port_d_return_t = port_d_return.pivot(index='daily_date', columns='momr', values='ret')

# Add prefix port in front of each column
port_d_return_t = port_d_return_t.add_prefix('P')

In [31]:
port_d_return_t.head()

momr,P1.0,P2.0,P3.0,P4.0,P5.0,P6.0,P7.0,P8.0,P9.0,P10.0
daily_date,,,,,,,,,,
1931-01-02,0.108129,0.084084,0.029738,0.062567,0.059118,0.031796,0.027509,0.011766,0.013190,0.017024
1931-01-03,0.017434,0.054541,0.037856,0.048063,0.036796,0.024025,0.015821,0.022562,0.010324,0.028963
1931-01-05,0.108850,0.021153,0.007968,-0.009274,-0.013565,-0.001715,0.003348,0.004877,-0.002438,0.002277
1931-01-06,0.011034,0.017633,0.032087,0.024398,0.015005,0.011491,0.013383,0.012327,0.010647,0.012261
1931-01-07,0.046308,0.033323,0.014989,0.010719,0.002536,0.023528,0.007031,0.015874,0.005673,0.014723


In [32]:
# Creating long-short portfolio
port_d_return_t = port_d_return_t.rename(columns={'P1.0':'losers', 'P10.0':'winners'})
port_d_return_t['long_short'] = port_d_return_t.winners - port_d_return_t.losers

In [33]:
port_d_return_t

momr,losers,P2.0,P3.0,P4.0,P5.0,P6.0,P7.0,P8.0,P9.0,winners,long_short
daily_date,,,,,,,,,,,
1931-01-02,0.108129,0.084084,0.029738,0.062567,0.059118,0.031796,0.027509,0.011766,0.013190,0.017024,-0.091106
1931-01-03,0.017434,0.054541,0.037856,0.048063,0.036796,0.024025,0.015821,0.022562,0.010324,0.028963,0.011529
1931-01-05,0.108850,0.021153,0.007968,-0.009274,-0.013565,-0.001715,0.003348,0.004877,-0.002438,0.002277,-0.106573
1931-01-06,0.011034,0.017633,0.032087,0.024398,0.015005,0.011491,0.013383,0.012327,0.010647,0.012261,0.001227
1931-01-07,0.046308,0.033323,0.014989,0.010719,0.002536,0.023528,0.007031,0.015874,0.005673,0.014723,-0.031585
...,...,...,...,...,...,...,...,...,...,...,...
2020-12-24,-0.005005,-0.000293,0.001712,0.003414,0.004666,0.004426,0.005906,0.002217,0.000417,-0.000541,0.004464
2020-12-28,0.001929,0.006508,0.004492,0.001054,0.002516,-0.000111,-0.000654,0.001083,-0.008104,-0.027763,-0.029692
2020-12-29,-0.005643,-0.004024,-0.002694,-0.003211,-0.002202,-0.004254,-0.004015,-0.006749,-0.005307,-0.008175,-0.002533


In [54]:
port_d_return_t.describe()

momr,losers,P2.0,P3.0,P4.0,P5.0,P6.0,P7.0,P8.0,P9.0,winners,long_short
count,23561.000000,23561.000000,23561.000000,23561.000000,23561.000000,23561.000000,23561.000000,23561.000000,23561.000000,23561.000000,23561.000000
mean,0.000463,0.000532,0.000546,0.000556,0.000528,0.000565,0.000561,0.000597,0.000674,0.000837,0.000375
std,0.015382,0.013005,0.012137,0.011396,0.011068,0.010949,0.010978,0.011333,0.012077,0.014622,0.012522
min,-0.168366,-0.137801,-0.172601,-0.162025,-0.176273,-0.171906,-0.184795,-0.195507,-0.188912,-0.220753,-0.215019
25%,-0.005430,-0.004299,-0.003964,-0.003770,-0.003715,-0.003670,-0.003820,-0.003861,-0.004147,-0.005066,-0.004180
50%,0.000415,0.000548,0.000649,0.000679,0.000763,0.000815,0.000878,0.000929,0.001064,0.001334,0.000787
75%,0.006007,0.005214,0.005032,0.004995,0.004973,0.005060,0.005144,0.005431,0.005927,0.007330,0.005640
max,0.247835,0.311769,0.289867,0.192753,0.229328,0.191896,0.241568,0.198902,0.193486,0.212556,0.114936


In [34]:
# Ensuring no NAN-values
#port_d_return_t['long_short'].isnull().sum()
port_d_return_t[port_d_return_t['long_short'].isnull()]

momr,losers,P2.0,P3.0,P4.0,P5.0,P6.0,P7.0,P8.0,P9.0,winners,long_short
daily_date,,,,,,,,,,,
1985-09-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-10-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
# Dropping NA values
port_d_return_t = port_d_return_t[port_d_return_t.index != '1985-09-27']
port_d_return_t = port_d_return_t[port_d_return_t.index != '2012-10-29']
port_d_return_t = port_d_return_t[port_d_return_t.index != '2017-09-30']

In [36]:
# Ensure no N/A
port_d_return_t['long_short'].isnull().sum()

0

In [37]:
port_d_return_t['losers'].isnull().sum()

0

In [38]:
port_d_return_t['winners'].isnull().sum()

0

In [39]:
# Defining long_short return
long_short_ret = port_d_return_t['long_short']
winners_ret = port_d_return_t['winners']
losers_ret = port_d_return_t['losers']

In [40]:
# Save as CSV
long_short_ret.to_csv('MOM_1931.csv', index=True)
winners_ret.to_csv('Winners_1931.csv', index=True)
losers_ret.to_csv('Losers_1931.csv', index=True)

### Daily market data

In [41]:
# Loading Fama French 3 Factors (only include rows with factors)
ff = pd.read_csv('FF3Fdaily.csv', skiprows = 3, nrows=24896, index_col = 0)

In [42]:
# Get in decimal format 
ff = ff / 100

In [43]:
# Setting datetime format
ff.index = pd.to_datetime(ff.index, format = '%Y%m%d')

In [44]:
# Results starting dates
ff = ff[ff.index >= start_date]

In [45]:
ff

,Mkt-RF,SMB,HML,RF
1931-01-02,0.0349,0.0007,0.0063,0.00006
1931-01-03,0.0182,-0.0026,0.0296,0.00006
1931-01-05,-0.0080,0.0028,0.0107,0.00006
1931-01-06,0.0151,-0.0026,0.0095,0.00006
1931-01-07,0.0005,0.0107,0.0103,0.00006
...,...,...,...,...
2020-12-24,0.0021,-0.0039,-0.0017,0.00000
2020-12-28,0.0046,-0.0072,0.0032,0.00000
2020-12-29,-0.0040,-0.0152,0.0024,0.00000
2020-12-30,0.0027,0.0099,0.0004,0.00000


In [46]:
# Testing if there is any errors 
ff['RF'] = pd.to_numeric(ff['RF'], errors='coerce') 
ff['Mkt-RF'] = pd.to_numeric(ff['Mkt-RF'], errors='coerce') 

ff['Mkt-RF'].isnull().sum()

0

In [47]:
# Calculating market return
ff['Mkt'] = ff['Mkt-RF'] + ff['RF']
market = ff['Mkt']

In [48]:
# Setting index to UTC (for it to work with Pyfolio)
market.index = pd.to_datetime(market.index).tz_localize('UTC')
market.index.name = 'date'

In [49]:
market

date
1931-01-02 00:00:00+00:00    0.03496
1931-01-03 00:00:00+00:00    0.01826
1931-01-05 00:00:00+00:00   -0.00794
1931-01-06 00:00:00+00:00    0.01516
1931-01-07 00:00:00+00:00    0.00056
                              ...   
2020-12-24 00:00:00+00:00    0.00210
2020-12-28 00:00:00+00:00    0.00460
2020-12-29 00:00:00+00:00   -0.00400
2020-12-30 00:00:00+00:00    0.00270
2020-12-31 00:00:00+00:00    0.00390
Name: Mkt, Length: 23561, dtype: float64

In [50]:
# To as CSV
market.to_csv('market_benchmark_1931.csv', index=True)

In [51]:
# Defining RF
risk_free = ff['RF']

In [52]:
# Setting index to UTC (for it to work with Pyfolio)
risk_free.index = pd.to_datetime(risk_free.index).tz_localize('UTC')
risk_free.index.name = 'date'

In [53]:
# To as CSV
risk_free.to_csv('risk_free_1931.csv', index=True)